In [1]:
clean_up = True
%run stdPackages.ipynb
ws = gams.GamsWorkspace(working_directory=d['work']) # specify where you want to run the GAMS models from (here the repository referred to in d['work'])
d['data'] = os.path.join(d['data'], 'IO2018')

The file _gams_py_gdb0.gdx is still active and was not deleted.
The file _gams_py_gdb1.gdx is still active and was not deleted.
The file _gams_py_gdb3.gdx is still active and was not deleted.


## SmallGR, Government sector

### Data/settings

Load data, settings:

In [2]:
name = 'smallGR' # name of model
module = name+'_G' # name of module
db_IO = GpyDB(pickle_path = os.path.join(d['data'], f'IO_{name}')) # load IO database named IO_name
with open(f"{d['data']}\\glob_{name}","rb") as file: # load global settings anmed glob_name
    glob=pickle.load(file)
dataFile = os.path.join(d['data'],'smallGR_Government.xlsx')

Beyond the standard IO data, we load relevant data for nesting structure on public consumption goods:

In [3]:
Tree = nestingTree.aggTree_from_data(dataFile, read_trees = {'gConsumption': {'name': 'gc', 'f':'CES'}}, name = module)()

### Balanced Budget Model

The goverment sector finances public consumption by levying taxes on households, domestic, and foreign firms (even itself). Thus, the government sector behaves in some ways as the simple households, with the exceptions that (1) it does not supply labor and (2) it does not have any intertemporal prefernences. Instead, it balances its budget in each period. 

In the small model here, we do not account for all taxes and subsidies. For simplicity, we add an adhoc adjustment term to the government budget that balances it in the initial period. This "j"-term is included permanently in the model. For all other time periods than this, a single tax instrument is devoted to balance the budget endogenously. The default instrument is the tax on labor income.

In [4]:
db_IO['TotalTax'] = adj.rc_pd(db_IO.get('TotalTax'), ('not', pd.Index(['itory'],name='s')))

Define the subset of sectors that pay taxes:

In [5]:
db_IO['d_TotalTax'] = db_IO['TotalTax'].index

Compute the need for readjustment in the initial year:

In [6]:
jG_budget = gpy(sum(db_IO.get('vD').xs('G'))-sum(db_IO.get('TotalTax')), name = 'jG_budget', **{'type':'parameter'})

Define $\tau_D$ to match taxes from IO data:

In [7]:
tVAT = pd.Series(db_IO.get('TotalTax').xs('G') / db_IO.get('qD').xs('G').sum(), index = adj.rc_pd(db_IO.get('qD'), Tree.get('s')).index, name = 'tauD')
gpyDB.add_or_merge_vals(db_IO, tVAT, 'tauD')

Export adjusted IO data again:

In [8]:
db_IO.export(repo=d['data'])

Add time index to the IO data:

In [9]:
def addT(symbol, t):
    return adjMultiIndex.bc(symbol, t).reorder_levels(['t']+symbol.index.names if 't' not in symbol.index.names else symbol.index.names)
[db_IO.__setitem__(k, addT(db_IO.get(k), glob.db['t'].vals)) for k in db_IO.getTypes(['variable','scalar_variable'])];

### Initialize module

*Initialize module without any savings first (static method):*

In [10]:
G = CGE_Government.balancedBudget(tree = Tree, glob = glob) # init module
G.s.db['labor'] = pd.MultiIndex.from_tuples([('HH','L')], names = ['s','n'])
G.s.db['d_TotalTax'] = db_IO.get('d_TotalTax')
G.s.db['jG_budget'] = jG_budget
G.s.db.update_alias(db_IO.get('alias_'))
G.s.db['tauD0'] = db_IO.get('tauD').rename('tauD0')
G.s.db['tauD'] = db_IO.get('tauD').copy()
G.s.db['TotalTax'] = db_IO.get('TotalTax').copy()

*Value shares for nesting part:*

In [11]:
v = valueShares.SimpleRamsey(Tree, db_IO.copy())
v.compile() # set up model structure, and make sure to initialize symbols if they are not yet defined in the database 
v.write(); # write GAMS code used for the model
m = v.run(exportTo = d['work'],ws=ws) # solve the "model".

Use value shares to initialize variables:
* Outputs and inputs are provided by IO data.
* For intermediate goods, assume a price of 1 (default option in the class, so we don't have to do anything) and set value share = quantity.
* Set share parameters to the ones identified by value share system.

In [12]:
gpyDB.GpyDBs_AOM_Second(G.s.db, gpy(adj.rc_pd(m.out_db.get('vD'), G.get('int').union(G.get('output'))).rename('qD'))) # set intermediate and output levels
gpyDB.GpyDBs_AOM_Second(G.s.db, gpy(m.out_db.get('mu').xs(G.get('t0')[0]).rename('mu'))) # set share parameters
G.s.db['a'] = db_IO.get('a')
aggregateDB.readSets(G.s.db)

Use sneaky calibration method to solve and calibrate:

In [13]:
G.compile(initDB=True)
G.s.setstate('B_standAlone')
G.write();
# m_base = G.run(exportTo = d['work'],ws=ws) # solve the "model".
# G.s['solve'] = f"""@SolveEmptyNLP({G.s['name']})""" # test run as NLP model
# G.write();
mCalib, cp = G.sneakyCalib(db_IO, cState = 'C_standAlone', ws = ws, loop_kwargs = {'n': 25}, **{'cns': 'CONOPT4'})

### 5. Export 

In [14]:
robust.robust_merge_dbs(G.s.db, mCalib.out_db.getTypes(['variable','scalar_variable']), priority = 'second')
# G.s.setstate('B')

Export:

In [15]:
with open(os.path.join(d['data'], G.name), "wb") as file:
    pickle.dump(G,file)